In [1]:
from typing import List, Tuple
from math import pi
from datetime import datetime
import rospy
from copy import deepcopy

from moveit_commander import PlanningSceneInterface

from geometry_msgs.msg import (
    Pose,
    PoseStamped,
    Point,
    Quaternion,
    Vector3,
)
from commander.msg import Goal
from commander.srv import (
    ExecuteTrajectory,
    PlanGoal,
    PlanGoalRequest,
    PlanSequence,
    PlanSequenceRequest,
    PickPlace,
    GetTcpPose,
    VisualizePoses,
    SetEe,
)
from industrial_reconstruction_msgs.srv import (
    StartReconstruction,
    StartReconstructionRequest,
    StopReconstruction,
    StopReconstructionRequest,
)

from commander.utils import poses_from_yaml, load_scene
from commander.transform_utils import orient_poses, create_rotation_matrix, apply_transformation

CAPTURE = True

rospy.init_node("reconstruction")

load_scene()

plan_goal_srv = rospy.ServiceProxy("commander/plan_goal", PlanGoal)
plan_sequence_srv = rospy.ServiceProxy("commander/plan_sequence", PlanSequence)
execute_trajectory_srv = rospy.ServiceProxy("commander/execute_trajectory", ExecuteTrajectory)
get_tcp_pose_srv = rospy.ServiceProxy("commander/get_tcp_pose", GetTcpPose)
set_ee_srv = rospy.ServiceProxy("commander/set_ee", SetEe)
pick_place_srv = rospy.ServiceProxy("commander/pick_place", PickPlace)

if CAPTURE:
    start_recon = rospy.ServiceProxy("/start_reconstruction", StartReconstruction)
    stop_recon = rospy.ServiceProxy("/stop_reconstruction", StopReconstruction)


def display_poses(poses: List[Pose], frame_id: str = "base_link") -> None:
    rospy.wait_for_service("/visualize_poses", timeout=10)
    visualize_poses = rospy.ServiceProxy("/visualize_poses", VisualizePoses)
    visualize_poses(frame_id, poses)


def gen_recon_msg(path: str) -> Tuple[StartReconstructionRequest, StopReconstructionRequest]:
    start_srv_req = StartReconstructionRequest()
    start_srv_req.tracking_frame = "rgb_camera_tcp"
    start_srv_req.relative_frame = "base_link"
    start_srv_req.translation_distance = 0.0
    start_srv_req.rotational_distance = 0.0
    start_srv_req.live = True
    start_srv_req.tsdf_params.voxel_length = 0.001
    start_srv_req.tsdf_params.sdf_trunc = 0.002
    start_srv_req.tsdf_params.min_box_values = Vector3(x=0.0, y=0.0, z=0.0)
    start_srv_req.tsdf_params.max_box_values = Vector3(x=0.0, y=0.0, z=0.0)
    start_srv_req.rgbd_params.depth_scale = 1
    start_srv_req.rgbd_params.depth_trunc = 1.0
    start_srv_req.rgbd_params.convert_rgb_to_intensity = False

    stop_srv_req = StopReconstructionRequest()
    path = path + datetime.now().strftime("%m_%d_%H_%M") + ".ply"
    stop_srv_req.mesh_filepath = path

    return start_srv_req, stop_srv_req

In [2]:
cam_home = [0, -pi / 2, pi / 2, 0, pi / 2, pi]
target0 = Pose(
    position=Point(0.7, -0.3, 0.35),
    orientation=Quaternion(0.7071067811865476, -0.7071067811865475, 0.0, 0.0),
)
target1 = Pose(
    position=Point(0.7, 0.3, 0.35),
    orientation=Quaternion(0.7071067811865476, -0.7071067811865475, 0.0, 0.0),
)
display_poses([target0, target1])
success = set_ee_srv("rgb_camera_tcp")

In [3]:
resp = plan_goal_srv(Goal(joint_values=cam_home, vel_scale=0.2, acc_scale=0.1, planner="ptp"))

In [4]:
execute_trajectory_srv()

success: True

In [10]:
yaml_file_path = "/dev_ws/src/commander/data/stupid_toolpath.yaml"
poses = poses_from_yaml(yaml_file_path)

origin = Pose(position=Point(1.3, 0.0, 0), orientation=Quaternion(0.0, 0.0, 0.0, 1.0))
poses = orient_poses(origin, poses)
tf = create_rotation_matrix([0, 0, pi])
poses = [apply_transformation(pose, tf) for pose in poses]
display_poses(poses)

In [13]:
goals = []
blends = []
for pose in poses:
    goal = Goal(pose=pose, vel_scale=0.1, acc_scale=0.1, planner="ptp")
    goals.append(goal)
    blends.append(0.05)

blends[0] = 0.0
blends[-1] = 0.0


In [ ]:
if CAPTURE:
    start_recon_req, stop_recon_req = gen_recon_msg("/home/huanyu/capture")

resp = plan_goal_srv(Goal(pose=cam_home, vel_scale=0.1, acc_scale=0.1, planner="ptp"))
if resp.success and not resp.configuration_change:
    print("Start recon")
    if CAPTURE:
        start_recon(start_recon_req)
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()

resp = plan_sequence_srv(goals, blends)
if resp.success:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()
if CAPTURE:
    stop_recon(stop_recon_req)